In [1]:
#Importar modulos necessarios

import json
from selenium import webdriver # selenium 4.20.0
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager # version 4.0.1
import pandas as pd
import time

from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from mplsoccer.pitch import Pitch
from scipy.stats import kde
from scipy.ndimage import gaussian_filter

from matplotlib.colors import ListedColormap

import requests

from mplsoccer import (VerticalPitch, Pitch, create_transparent_cmap,
                       FontManager, arrowhead_marker, Sbopen)   

from pathlib import Path
import os


from datetime import datetime
import pytz

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
#Mostrando todas as linhas e colunas do DF
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
#Inserir link do time a ser analisado
team_link = "https://www.sofascore.com/team/football/internacional/1966"
#Inserir pasta em que serão salvos os arquivos
save_path = 'C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/'

In [4]:
#Esta funçao padrão cria o dataframe para cada um dos tipos de estatisticas da partida 
def add_match_stats(jj, kk, ii, tt):
    match_stats_column.append(match_stats['statistics'][jj]['groups'][kk]['statisticsItems'][ii]['name'])
    match_stats_home.append(match_stats['statistics'][jj]['groups'][kk]['statisticsItems'][ii]['home'])
    match_stats_away.append(match_stats['statistics'][jj]['groups'][kk]['statisticsItems'][ii]['away'])
    #Verificando se a estatistica que vamos tentar colocar já existe na tabela, se existir será adicionado na proxima coluna na respectiva linha
    if (match_stats_column[ii] in match_stats_df['MO_stats'].values) == True:
        ind = match_stats_df.index[match_stats_df['MO_stats']==match_stats_column[ii]]
        match_stats_df.loc[ind, (tt+1)] = match_stats_home[ii]
        match_stats_df.loc[ind, (tt+2)] = match_stats_away[ii]
    return(match_stats_df)

In [5]:
#Salvando dados da partida em CSV
def match_df_to_csv(dataframe, data_type):
    filepath = save_path+data_type+'/'
    print(filepath)
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    filename = dataframe.name[:-3]+'_'+time_stamp_path+'_'+str(match_id)+'.csv'
    file_in_csv = dataframe.to_csv(Path(filepath + filename), index=False)
    return(file_in_csv)

In [6]:
#Pegando heatmap de todos os jogadores do time
def create_heatmap_dict(match_id,players_stats_df):
    heatmap_dict = {}
    heatmap_dict[str(match_id)+'_heatmap'] = {}
    for i in range(len(players_stats_df.columns)):
        if i != 0:
            row = players_stats_df.index[players_stats_df[0] == 'id'].tolist()[0]
            player_id = players_stats_df[i][row]
            
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            session.mount('https://', adapter)

            player_heatmap = json.loads(session.get('https://www.sofascore.com/api/v1/event/'+str(match_id)+'/player/'+str(player_id)+'/heatmap').text)
            heatmap_dict[str(match_id)+'_heatmap'][str(player_id)] = player_heatmap
    return (heatmap_dict)

In [7]:
#Salvando o heatmap dos jogadores em CSV
def csv_player_heatmap(heatmap_dict, player_id):
    player_heatmap = heatmap_dict[str(match_id)+'_heatmap'][str(player_id)]
    #Transformando dicionario em dataframe e depois separando a coluna heatmap em duas, uma para coordenada x e outra para y
    player_heatmap_df = pd.DataFrame.from_dict(player_heatmap)
    player_heatmap_coord_df = pd.json_normalize(player_heatmap_df['heatmap'])
    filepath = save_path+str(player_id)+'/'
    print(filepath)
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    filename = 'heatmap_coord_'+'_'+str(player_id)+'_'+time_stamp_path+'_'+str(match_id)+'.csv'
    coord_to_csv = player_heatmap_coord_df.to_csv(Path(filepath + filename), index=False)
    return(coord_to_csv)

In [8]:
with open(save_path+"/brasileirao_58766.txt") as f:
     match_links_txt = [line.rstrip('\n') for line in f]

In [9]:
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)
options.add_argument("--start-maximized")
options.add_argument("--headless=new")

service = ChromeService(ChromeDriverManager().install())

In [10]:
#Usando webdriver para abrir página do time
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_page_load_timeout(20)
try:
    driver.get(team_link)
except:
    pass

#Time a ser analisado
team_id = team_link.rsplit('/', 1)[1]
driver.close()

In [11]:
for lnk in range(len(match_links_txt)):
    match_link = match_links_txt[lnk]
    print(lnk)   
    #Abrindo página no webdriver do jogo escolhido
    start = time.time()
    try:
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(match_link)
    except:
        pass
    end = time.time()    
    print("Para abrir o link levou ", end-start, 'segundos')
    start = time.time()
    #Aqui é reduzido o zoom da pagina e rolado até o final para poder aparecer todo o conteúdo
    driver.execute_script("document.body.style.zoom='50%'") 
    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    end = time.time()
    print("Para reduzir zoom e rolar para baixo levou", end-start, "segundos")
    #Aqui está sendo retirado o id da partida de acordo com o link fornecido
    match_id = match_link.rsplit(':', 1)[1]
    start = time.time()
    #Aqui estão sendo extraídos todos os logs que estão sendo carregados na página, um deles contém o API que queremos
    logs_raw = driver.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]
    
    #Aqui, como tinha mais de um logs com o nome lineups que queremos, e só um tinha os dados que precisamos, foi feito condicionais
    #para poder conseguir o requestId para a requisição feita pelo website para obter os dados de lineups
    for x in logs:
        if 'lineups' in x['params'].get('headers', {}).get(':path', ''):
            if 'api'in x['params'].get('headers', {}).get(':path', ''):
                if x['params'].get('headers', {}).get(':method')=='GET':
                    match_lineups_id = x['params']
        if 'statistics' in x['params'].get('headers', {}).get(':path', ''):
            if 'api'in x['params'].get('headers', {}).get(':path', ''):
                if x['params'].get('headers', {}).get(':method')=='GET':
                    match_stats_id = x['params']
        if 'shotmap' in x['params'].get('headers', {}).get(':path', ''):
            if 'api'in x['params'].get('headers', {}).get(':path', ''):
                if x['params'].get('headers', {}).get(':method')=='GET':
                    match_shotmap_id = x['params']                
        if match_id in x['params'].get('headers', {}).get(':path', ''):
            if ('/api/v1/event/'+match_id) == x['params'].get('headers', {}).get(':path', ''):
                if x['params'].get('headers', {}).get(':method')=='GET':
                    match_info_id = x['params']
    end = time.time()
    print("Para pegar o ID de cada API levou", end-start, "segundos")                
    #Carregando as informações do API através de um json. Cada um abaixo é um dict com as informações que queremos. Foi necessário incluir um try
    #porque em algumas partidas não havia dados, e estava salvando os CSV das partidas anteriores. Dessa forma só vai salvar arquivos da partida
    #se houver dados daquela partida.
    start = time.time()
    try:
        match_info = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': match_info_id["requestId"]})['body'])
        ML=1
    except:
        ML = 0
        pass
    if ML==1:
        #Criando condição para saber se o time com o ID fornecido no começo é mandante ou visitante
        if str(match_info["event"]["homeTeam"]['id']) == team_id:
            team_side = 'home'
            opponent_side = 'away'
        else:
            team_side = 'away'
            opponent_side = 'home' 
        time_stamp_path = 'T'+str(match_info['event']['startTimestamp'])
        time_stamp = match_info['event']['startTimestamp']
        timezone = pytz.timezone('America/Sao_Paulo')
        match_date = str((datetime.fromtimestamp(time_stamp, tz=timezone)).date())
    else:
        pass                    
    end = time.time()
    print("Para ver se o time é mandante e pegar o dia do jogo levou", end-start, "segundos")    
    start = time.time()
    try:
        match_lineups = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': match_lineups_id["requestId"]})['body'])
        ML = 2
    except:
        ML = 0
        pass
    if ML==2:    
        #Criando a lista com as estatisticas dos jogadores
        #Criando lista par colocar o dado lido de cada jogador e depois colocar no dataframe
        player_zero = []
        #Criando lista para variáveis que não serão usadas
        variavel_ruim = []
        #Para cada jogador dentro da lista de jogadores do time da casa
        for i in range(len(match_lineups[team_side]['players'])):
            #Se o i é menor que 1, é porque estamos falando do primeiro jogador da lista, e ainda não temos nenhuma informação no DF
            if i < 1:
                #Eliminando dicionario que tinha varios nomes para o pais de origem do jogador
                try:
                    match_lineups[team_side]['players'][i]['player']['country'] = match_lineups[team_side]['players'][i]['player']['country']['name']
                except:
                    match_lineups[team_side]['players'][i]['player']['country'] = None
                #Eliminando o dicionario das notas dos jogadores, deixando apenas a nota original
                try:
                    match_lineups[team_side]['players'][i]['statistics']['ratingVersions'] = match_lineups[team_side]['players'][i]['statistics']['ratingVersions']['original']
                except:
                    match_lineups[team_side]['players'][i]['statistics']['ratingVersions'] = None
                #Separando cada parte das informações do jogador 0 e transformando em lista
                player_0 = (match_lineups[team_side]['players'][i])
                player_0_info = list(match_lineups[team_side]['players'][i]['player'].items())
                player_0_stats = list(match_lineups[team_side]['players'][i]['statistics'].items())
                player_0_info_alt = list(match_lineups[team_side]['players'][i].items())[1:-1]
                #Juntando todas as infos do jogador 0 numa lista só
                player_zero = player_0_info + player_0_info_alt + player_0_stats
                header = player_zero[0]
                players_stats_df = pd.DataFrame(player_zero)
            #Se i >=1 é porque não é mais o primeiro jogador, e já temos informações na primeira coluna e por isso temos que verificar se a informação
            #do jogador seguinte já existe na primeira coluna ou é uma variável nova
            else :
                try:
                    match_lineups[team_side]['players'][i]['player']['country'] = match_lineups[team_side]['players'][i]['player']['country']['name']
                except:
                    match_lineups[team_side]['players'][i]['player']['country'] = None
                try:
                    match_lineups[team_side]['players'][i]['statistics']['ratingVersions'] = match_lineups[team_side]['players'][i]['statistics']['ratingVersions']['original']
                except:
                    match_lineups[team_side]['players'][i]['statistics']['ratingVersions'] = None
                player_0 = (match_lineups[team_side]['players'][i])
                player_0_info = list(match_lineups[team_side]['players'][i]['player'].items())
                player_0_stats = list(match_lineups[team_side]['players'][i]['statistics'].items())
                player_0_info_alt = list(match_lineups[team_side]['players'][i].items())[1:-1]
                player_zero = player_0_info + player_0_info_alt + player_0_stats
                header = player_zero[0]
                #Para cada informação na lista do jogador, será verificado se já está na primeira coluna ou não
                for t in range(0,len(player_zero)):
                    #Se já estiver na primeira coluna, será apenas localizado qual o indice (linha) pertence a informaçao e será armazenada nela para o
                    #respectivo jogador
                    if (player_zero[t][0] in players_stats_df[0].values) == True:
                        ind = players_stats_df.index[players_stats_df[0]==player_zero[t][0]]
                        players_stats_df.loc[ind, (i+1)] = player_zero[t][1]
                    #Se não, quer dizer que é um informação nova e deve ser criada uma nova linha para poder armazenar a informação
                    else :
                        #adicionar linha com esse string
                        #Verificando se é uma das variáveis ruins, nomes em árabe que não são interessantes
                        if type(player_zero[t][1]) is dict:
                            variavel_ruim.append(player_zero[t][1]) #posso excluir isso depois, apenas aqui para verificar quais são as variaveis em dict
                        #Se não for dicionario, pode seguir normalmente
                        else:
                            ind = len(players_stats_df)
                            players_stats_df.loc[ind, (i+1)] = player_zero[t][1]
                            players_stats_df.loc[ind, 0] = str(player_zero[t][0])

        #Excluindo linhas de firstName e lastName que na maioria estão vazias
        ind = players_stats_df.index[players_stats_df[0]=='firstName']
        players_stats_df = players_stats_df.drop(ind)
        ind = players_stats_df.index[players_stats_df[0]=='lastName']
        players_stats_df = players_stats_df.drop(ind)
        #Preenchendo todos os NaN com 0 e resetando a ordem dos indices, pois depois de remover as linhas no codigo acima
        #perdeu-se a ordem dos indices
        players_stats_df = players_stats_df.fillna(0)
        players_stats_df = players_stats_df.reset_index(drop=True)
        players_stats_df.name = 'players_stats_df'
        #Atribuindo o nome de heatmap_dict para o dicionario do heatmap criado pela funçao create_heatmap_dict
        heatmap_dict = create_heatmap_dict(match_id,players_stats_df)
        #Tentar tirar apenas pelo dict, sem usar o dataframe
        for i in range(len(heatmap_dict[str(match_id)+'_heatmap'])):
            row = players_stats_df.index[players_stats_df[0] == 'id'].tolist()[0]
            player_id = players_stats_df[i+1][row]
            if list(heatmap_dict[str(match_id)+'_heatmap'][str(player_id)].items())[0][0] != 'error':
                csv_player_heatmap(heatmap_dict, player_id) 
        match_df_to_csv(players_stats_df,'players_stats')

        #Inserir nome dos jogadores na lista caso ainda não estejam
        if lnk<1:
            for var in range(len(players_stats_df.columns)):
                team_lineup = []
                team_lineup.append(players_stats_df[var][7])
                team_lineup.append(players_stats_df[var][0])
                team_lineup.append(players_stats_df[var][3])
                if var<1:
                    team_lineup_df = pd.DataFrame(team_lineup)
                else:
                    team_lineup_df.loc[0, (var)] = str(team_lineup[0])
                    team_lineup_df.loc[1, (var)] = team_lineup[1]
                    team_lineup_df.loc[2, (var)] = str(team_lineup[2])
            #Criando tabela com os dados para cada partida
            matches_info = []
            if team_side == 'home':
                if match_info['event']['winnerCode'] == 1:
                    match_result = 'W'
                if match_info['event']['winnerCode'] == 2:
                    match_result = 'L' 
                if match_info['event']['winnerCode'] == 3:
                    match_result = 'D'
            else:
                if match_info['event']['winnerCode'] == 1:
                    match_result = 'L'
                if match_info['event']['winnerCode'] == 2:
                    match_result = 'W'
                if match_info['event']['winnerCode'] == 3:
                    match_result = 'D'
            matches_info= [match_info['event']['startTimestamp'], match_id, match_info['event']['roundInfo']['round'], team_side, match_result]
            matches_info_df = pd.DataFrame(matches_info).transpose()
            matches_info_df.columns = ['Date', 'MatchID', 'Round', 'Teamside', 'Result']
        else:
            for var in range(len(players_stats_df.columns)):
                team_lineup = []
                team_lineup.append(players_stats_df[var][7])
                team_lineup.append(players_stats_df[var][0])
                team_lineup.append(players_stats_df[var][3])
                len_players_list = len(team_lineup_df.columns)
                if var<1:
                    pass
                else:
                    if str(team_lineup[0]) in team_lineup_df.iloc[0].values:
                        pass
                    else:
                        team_lineup_df.loc[0, (len_players_list)] = str(team_lineup[0])
                        team_lineup_df.loc[1, (len_players_list)] = team_lineup[1]
                        team_lineup_df.loc[2, (len_players_list)] = str(team_lineup[2])
            #Com a tabela de informações das partidas já criada, para cada partida vai ser adicionada uma nova linha            
            matches_info = []
            if team_side == 'home':
                if match_info['event']['winnerCode'] == 1:
                    match_result = 'W'
                if match_info['event']['winnerCode'] == 2:
                    match_result = 'L' 
                if match_info['event']['winnerCode'] == 3:
                    match_result = 'D'
            else:
                if match_info['event']['winnerCode'] == 1:
                    match_result = 'L'
                if match_info['event']['winnerCode'] == 2:
                    match_result = 'W'
                if match_info['event']['winnerCode'] == 3:
                    match_result = 'D' 
            matches_info_df.loc[len(matches_info_df)] = [match_info['event']['startTimestamp'], match_id, match_info['event']['roundInfo']['round'], team_side, match_result]
    else:
        pass    
    end = time.time()
    print("Para pegar os dados dos jogadores, dados da partida levou", end-start, "segundos")    
    start = time.time()
    try:
        match_stats = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': match_stats_id["requestId"]})['body'])
        ML = 3
    except:
        ML = 0
        pass 
    if ML ==3:
        #Pegando estatisticas da partida
        for k in range(len(match_stats['statistics'][0]['groups'])):
            match_stats_date = []
            match_stats_column = []
            match_stats_home = []
            match_stats_away = []
            #Cara cada uma das estatistica dentro do MatchOverview, vai ser adicionado o nome da estatistica e seu valor para mandante e visitante em cada linha
            j = 0
            for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                match_stats_date = match_date
                match_stats_column.append(match_stats['statistics'][j]['groups'][k]['statisticsItems'][i]['name'])
                match_stats_home.append(match_stats['statistics'][j]['groups'][k]['statisticsItems'][i]['home'])
                match_stats_away.append(match_stats['statistics'][j]['groups'][k]['statisticsItems'][i]['away'])
            #Gerar o dataframe e depois colocar as colunas com os valores para o ALL de mandante e visitante

            match_stats_df = pd.DataFrame({'MO_stats' : match_stats_column})
            match_stats_df['ALL_home'] = match_stats_home
            match_stats_df['ALL_away'] = match_stats_away
            match_stats_df.name = 'match_stats_df'
            
            if k==0: 
                #Mesmo processo para estatisticas do primeiro tempo
                #Pegar o numero de colunas que ja existe na tabela para que o proximo seja colocado ao lado da ultima coluna
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    MO_stats_df = match_stats_df
                #Mesmo processo para estatisticas do segundo tempo
                #Pegar o numero de colunas que ja existe na tabela para que o proximo seja colocado ao lado da ultima coluna
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = [] 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    MO_stats_df = match_stats_df
                match_stats_df['Date'] = match_stats_date
                MO_stats_df.loc[len(MO_stats_df), MO_stats_df.columns] = 'Goals',match_info['event']['homeScore']['normaltime'],match_info['event']['awayScore']['normaltime'],match_info['event']['homeScore']['period1'], match_info['event']['awayScore']['period1'],match_info['event']['homeScore']['period2'],match_info['event']['awayScore']['period2'], match_stats_date
                if team_side == 'home':
                    MO_stats_df.columns = ['MO_stats', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    MO_stats_df.columns = ['MO_stats', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
                    
            if k==1:
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    shots_df = match_stats_df
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    shots_df = match_stats_df
                shots_df['Date'] = match_stats_date
                if team_side == 'home':
                    shots_df.columns = ['Shots', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    shots_df.columns = ['Shots', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
        
            if k==2: 
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    attack_df = match_stats_df
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    attack_df = match_stats_df
                attack_df['Date'] = match_stats_date
                if team_side == 'home':
                    attack_df.columns = ['Attack', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    attack_df.columns = ['Attack', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
        
            if k==3:
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    passes_df = match_stats_df
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    passes_df = match_stats_df
                passes_df['Date'] = match_stats_date
                if team_side == 'home':
                    passes_df.columns = ['Passes', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    passes_df.columns = ['Passes', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
                
            if k==4:
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    duels_df = match_stats_df
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    duels_df = match_stats_df
                duels_df['Date'] = match_stats_date
                if team_side == 'home':
                    duels_df.columns = ['Duels', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    duels_df.columns = ['Duels', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
        
            if k==5:
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    defending_df = match_stats_df
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    defending_df = match_stats_df
                defending_df['Date'] = match_stats_date
                if team_side == 'home':
                    defending_df.columns = ['Defending', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    defending_df.columns = ['Defending', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
        
            if k==6:
                j = 1
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    goal_keeping_df = match_stats_df
                j = 2
                t = len(match_stats_df.columns)
                match_stats_column = []
                match_stats_home = []
                match_stats_away = []                 
                for i in range(len(match_stats['statistics'][j]['groups'][k]['statisticsItems'])):
                    add_match_stats(j, k, i, t)
                    goal_keeping_df = match_stats_df
                goal_keeping_df['Date'] = match_stats_date
                if team_side == 'home':
                    goal_keeping_df.columns = ['GoalKeeping', 'ALL_Inter', 'ALL_Opponent', '1ST_Inter', '1ST_Opponent', '2ND_Inter', '2ND_Opponent', 'Date']
                else:
                    goal_keeping_df.columns = ['GoalKeeping', 'ALL_Opponent', 'ALL_Inter', '1ST_Opponent', '1ST_Inter', '2ND_Opponent', '2ND_Inter', 'Date']
        end = time.time()
        print("Para pegar as estatisticas da partida levou", end-start, "segundos")
        start = time.time()
        #Atribuindo 0 para as células que ficaram NA e atribuindo o nome para os dataframe para que na hora de criar o csv, o nome do arquivo csv seja criado de acordo com o nome atribuido ao dataframe
        MO_stats_df = MO_stats_df.fillna(0)
        MO_stats_df.name = 'MO_stats_df'
        shots_df = shots_df.fillna(0)
        shots_df.name = 'shots_df'
        attack_df = attack_df.fillna(0)
        attack_df.name = 'attack_df'
        passes_df = passes_df.fillna(0)
        passes_df.name = 'passes_df'
        duels_df = duels_df.fillna(0)
        duels_df.name = 'duels_df'
        defending_df = defending_df.fillna(0)
        defending_df.name = 'defending_df'
        goal_keeping_df = goal_keeping_df.fillna(0)
        goal_keeping_df.name = 'goal_keeping_df'
        match_df_to_csv(MO_stats_df, 'MO_stats')
        match_df_to_csv(shots_df, 'shots')
        match_df_to_csv(attack_df, 'attack')
        match_df_to_csv(passes_df, 'passes')
        match_df_to_csv(duels_df, 'duels')
        match_df_to_csv(defending_df, 'defending')
        match_df_to_csv(goal_keeping_df, 'goal_keeping')
    else:
        pass    
    end = time.time()
    print("Para transoformar os df em csv levou", end-start, "segundos")
    start = time.time()
    try:
        match_shotmap = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': match_shotmap_id["requestId"]})['body'])
        #Criando lista para colocar o dado lido de cada chute e depois colocar no dataframe
        shotmap_zero = []
        
        #Para cada chute dentro da lista
        for i in range(len(match_shotmap['shotmap'])):
            #Se o i é menor que 1, é porque estamos falando do primeiro jogador da lista, e ainda não temos nenhuma informação no DF
            if i < 1:
                #Separando cada parte das informações do jogador que deu o chute e transformando em lista
                shotmap_player_info = list(match_shotmap['shotmap'][i]['player'].items())
                #Separando cada parte das estatisticas do chute do jogador e transformando em lista
                shotmap_stats = list(match_shotmap['shotmap'][i].items())
                #Removendo o id que aparece no item dentro de (match_shotmap['shotmap'][i].items()) pois ele estava confundindo com o id do jogador
                shotmap_stats = list(filter(lambda x: x[0] != "id", shotmap_stats))
                #Separando as coordenadas do chute do jogador e transformando em lista
                shotmap_coord = list(match_shotmap['shotmap'][i]['draw'].items())
                #Removendo os dados do jogador, pois já foi pego na lista shotmap_player_info
                del shotmap_stats[0]
                #Juntando todas as infos do jogador 0 numa lista só
                shotmap_zero = shotmap_player_info + shotmap_stats + shotmap_coord
                header = shotmap_zero[0]
                shotmap_df = pd.DataFrame(shotmap_zero)
                
            #Se i >=1 é porque não é mais o primeiro jogador, e já temos informações na primeira coluna e por isso temos que verificar se a informação
            #do jogador seguinte já existe na primeira coluna ou é uma variável nova
            else :
                #Separando cada parte das informações do jogador 0 e transformando em lista
                shotmap_player_info = list(match_shotmap['shotmap'][i]['player'].items())
                shotmap_stats = list(match_shotmap['shotmap'][i].items())
                shotmap_stats = list(filter(lambda x: x[0] != "id", shotmap_stats))
                shotmap_coord = list(match_shotmap['shotmap'][i]['draw'].items())
                del shotmap_stats[0]
                #Juntando todas as infos do jogador 0 numa lista só
                shotmap_zero = shotmap_player_info + shotmap_stats + shotmap_coord
                header = shotmap_zero[0]
                #Para cada informação na lista do jogador, será verificado se já está na primeira coluna ou não
                for t in range(0,len(shotmap_zero)):
                    #Se já estiver na primeira coluna, será apenas localizado qual o indice (linha) pertence a informaçao e será armazenada nela para o
                    #respectivo jogador
                    if (shotmap_zero[t][0] in shotmap_df[0].values) == True:
                        ind = shotmap_df.index[shotmap_df[0]==shotmap_zero[t][0]]
                        #Foi necessário diferencias se a estatistica está armazenada como dict, pois se for dict precisa ter [dict]
                        if type(shotmap_zero[t][1]) == dict:
                            shotmap_df.loc[ind, (i+1)] = [shotmap_zero[t][1]]
                        else:
                            shotmap_df.loc[ind, (i+1)] = shotmap_zero[t][1]
                    #Se não, quer dizer que é um informação nova e deve ser criada uma nova linha para poder armazenar a informação
                    else :
                        ind = len(shotmap_df)
                        #Foi necessário diferencias se a estatistica está armazenada como dict, pois se for dict precisa ter [dict]
                        if type(shotmap_zero[t][1]) == dict:
                            shotmap_df.loc[ind, (i+1)] = [shotmap_zero[t][1]]
                        else:
                            shotmap_df.loc[ind, (i+1)] = shotmap_zero[t][1]
                        shotmap_df.loc[ind, 0] = str(shotmap_zero[t][0])
        
        shotmap_df = shotmap_df.fillna(0)
        shotmap_df.name = 'shotmap_df'   
        match_df_to_csv(shotmap_df)
    except:
        pass
    end = time.time()
    print("Para salvar o dataframe de shotmap levou", end-start, "segundos")
    driver.close()
team_lineup_df.name = 'team_lineup'
match_df_to_csv(team_lineup_df, 'team_lineup')
matches_info_df.name = 'matches_info'
match_df_to_csv(matches_info_df, 'matches_info')

0
Para abrir o link levou  7.4820637702941895 segundos
Para reduzir zoom e rolar para baixo levou 6.261624336242676 segundos
Para pegar o ID de cada API levou 0.2078711986541748 segundos
Para ver se o time é mandante e pegar o dia do jogo levou 0.25960254669189453 segundos
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/581060/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/850023/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/876625/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/41837/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/243113/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/986233/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/358550/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/995071/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/247483/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/560116/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/1199435/
C:/Users/gusta/OneDrive/Documentos/Data Analysis v11/12

A partir daqui, são funções para visualizar alguns dados como shotmap de jogadores, ou mapa de calor de jogadores específicos.

In [12]:
def shotmap_player_tab(player_name):
    shotmap_player_df = pd.DataFrame(shotmap_df.loc[:,0].tolist())
    for i in range(len(shotmap_df.columns)):
        if shotmap_df.loc[0,i] == player_name:
            shotmap_player_df = pd.concat([shotmap_player_df, shotmap_df.loc[:,i]], axis=1)
    start_list = pd.DataFrame((shotmap_player_df.loc[25].tolist()[1:]))
    end_list = pd.DataFrame((shotmap_player_df.loc[26].tolist()[1:]))
    goal_list = pd.DataFrame((shotmap_player_df.loc[27].tolist()[1:]))
    xg_list = pd.DataFrame((shotmap_player_df.loc[16].tolist()[1:]))
    pitch = VerticalPitch(pad_bottom=0.5,  # pitch extends slightly below halfway line
                          half=True,  # half of a pitch
                          goal_type='box',
                          goal_alpha=0.8,
                          axis=True,
                          label = True)  # control the goal transparency
    fig, ax = pitch.draw(figsize=(8, 6))
    for i in range(1,len(shotmap_player_df.columns)):
        c_id = shotmap_player_df.columns[i]
        if shotmap_player_df.loc[9,c_id] =='goal':
            pitch.scatter((120-(start_list.y[i-1]*60/100)), start_list.x[i-1]*80/100,
                        # size varies between 100 and 1900 (points squared)
                        s=(xg_list[0][i-1] * 1900) + 100,
                        edgecolors='#b94b75',
                        linewidths=0.6,
                        c='white',
                        marker='football',
                        ax=ax)
        else:
            pitch.scatter((120-(start_list.y[i-1]*60/100)), start_list.x[i-1]*80/100,
                        # size varies between 100 and 1900 (points squared)
                        s=(xg_list[0][i-1] * 1900) + 100,
                        edgecolors='#b94b75',  # give the markers a charcoal border
                        c='None',  # no facecolor for the markers
                        hatch='///',  # the all important hatch (triple diagonal lines)
                        # for other markers types see: https://matplotlib.org/api/markers_api.html
                        marker='o',
                        ax=ax)
    
    plt.show()            
    #return(shotmap_player_df)

In [13]:
def print_player_heatmap(player_id):
    player_heatmap = create_heatmap_dict(match_id,players_stats_df)[str(match_id)+'_heatmap'][str(player_id)]
    #Transformando dicionario em dataframe e depois separando a coluna heatmap em duas, uma para coordenada x e outra para y
    player_heatmap_df = pd.DataFrame.from_dict(player_heatmap)
    player_heatmap_coord_df = pd.json_normalize(player_heatmap_df['heatmap'])
    #Transformando as coordenadas que vieram do API e estão em porcentagem do tamanho do campo para uma coordenada de posição no campo
    #considerado do mplsoccer, que possui tamanho 120 x 80.
    x = player_heatmap_coord_df['x'] * 120/100
    y = player_heatmap_coord_df['y'] * 80/100
    #Passando de dataframe para array
    x = x.to_numpy()
    y = y.to_numpy()
    #É necessario inverter o eixo vertical pois o mpl soccer considera a origem
    #começando no canto superior esquerdo e as coordenadas do API vêm do cantos esquerdo inferior
    #Aqui é feito um cálculo da densidade dos pontos, em que é calculado que a maior distribuição está em determinado ponto e então plotado em função disso
    pitch = Pitch(pitch_color='grass', line_color='white',
                  stripe=True, axis=True, label=True, tick=True)  # optional stripes
    fig, ax = pitch.draw(figsize=(8, 4))
    sns.kdeplot(y=y, x=x,fill=True, ax=ax, cmap='YlOrRd',
                  shade_lowest=False,levels=10,kernel='gau',gridsize=50,
                  bw='scott', bw_adjust=0.55, alpha = 0.8)
    ax.invert_yaxis()

In [14]:
#print_player_heatmap()

In [15]:
#shotmap_player_tab('')